In [1]:
import numpy as np
import os
import cv2
import csv
import xmltodict
import matplotlib.pyplot as plt
import random

In [2]:
WIDTH = 640
HEIGHT = 480

#product_list = set(['extinguisher', 'backpack', 'drill'])
product_list = set(['barcode'])

## Create Mask from XML File

In [3]:
def xml2mask(xml_path):
    """Convert xml file to a label image
    Read a xml file, and generate a gray image.
    There are only three different values in a image:
    255: object
    128: barcode
    0: background
    Args:
        xml_path: xml path
    Returns:
        1-channel lable image
    """
    global product_list
    file_exist = os.path.isfile(xml_path)    # True
    mask = np.zeros([HEIGHT, WIDTH], dtype = np.uint8)
    if file_exist:
        # print xml_path
        with open(xml_path) as fd:
            label_dict = xmltodict.parse(fd.read())

    else:
        print ("This path doesn't exist")
        # print xml_path
        return mask
    if 'object' in label_dict['annotation']:
        if type(label_dict['annotation']['object']).__name__ != "list":
            try:
                object_ = label_dict['annotation']['object']
                '''if object_['name'] == "extinguisher" and object_['deleted'] == '0':
                    poly_vertice = []
                    for pts_idx in object_['polygon']['pt']:
                        poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                    poly_vertice = np.array(poly_vertice, np.int32)
                    cv2.fillConvexPoly(mask, poly_vertice, 1)
                if object_['name'] == "backpack" and object_['deleted'] == '0':
                    poly_vertice = []
                    for pts_idx in object_['polygon']['pt']:
                        poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                    poly_vertice = np.array(poly_vertice, np.int32)
                    cv2.fillConvexPoly(mask, poly_vertice, 2)
                if object_['name'] == "drill" and object_['deleted'] == '0':
                    poly_vertice = []
                    for pts_idx in object_['polygon']['pt']:
                        poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                    poly_vertice = np.array(poly_vertice, np.int32)
                    cv2.fillConvexPoly(mask, poly_vertice, 3)'''
                if object_['name'] == "barcode" and object_['deleted'] == '0':
                    poly_vertice = []
                    for pts_idx in object_['polygon']['pt']:
                        poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                    poly_vertice = np.array(poly_vertice, np.int32)
                    cv2.fillConvexPoly(mask, poly_vertice, 3)
            except Exception as e:
                print (e)
        else:
            try:
                for object_ in label_dict['annotation']['object']:
                    # print (object_['name'])
                    '''if object_['name'] == "extinguisher" and object_['deleted'] == '0':
                        poly_vertice = []
                        for pts_idx in object_['polygon']['pt']:
                            poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                        poly_vertice = np.array(poly_vertice, np.int32)
                        cv2.fillConvexPoly(mask, poly_vertice, 1)
                    if object_['name'] == "backpack" and object_['deleted'] == '0':
                        poly_vertice = []
                        for pts_idx in object_['polygon']['pt']:
                            poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                        poly_vertice = np.array(poly_vertice, np.int32)
                        cv2.fillConvexPoly(mask, poly_vertice, 2)
                    if object_['name'] == "drill" and object_['deleted'] == '0':
                        poly_vertice = []
                        for pts_idx in object_['polygon']['pt']:
                            poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                        poly_vertice = np.array(poly_vertice, np.int32)
                        cv2.fillConvexPoly(mask, poly_vertice, 3)'''
                    if object_['name'] == "barcode" and object_['deleted'] == '0':
                        poly_vertice = []
                        for pts_idx in object_['polygon']['pt']:
                            poly_vertice.append([int(pts_idx['x']), int(pts_idx['y'])])
                        poly_vertice = np.array(poly_vertice, np.int32)
                        cv2.fillConvexPoly(mask, poly_vertice, 1)
            except Exception as e:
                print (e)
    return mask

## Generate Mask

### For SubT dataset

In [4]:
datapath = "/media/arg_ws3/5E703E3A703E18EB/data/subt_tunnel_hsinchu/"
bags = ['bag1', 'bag2', 'bag3']
file_list = []
for bag in bags:
    path = os.path.join(datapath, bag, 'Annotations')

    for (dirpath, dirnames, filenames) in os.walk(path):
        #print(filenames)
        for filename in filenames:
            mask_dir = dirpath.replace("Annotations", "Masks")
            if not os.path.exists(mask_dir):
                os.makedirs(mask_dir)
            mask_name = filename.split(".")[0] + ".png"
            img_path = os.path.join(dirpath, filename)
            mask_path = os.path.join(mask_dir, mask_name)
            mask = xml2mask(img_path)
            cv2.imwrite(mask_path, mask,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
            file_list.append([img_path[len(datapath):], mask_path[len(datapath):]])

### For MM barcode dataset (UNDO)

In [ ]:
datapath = "/media/arg_ws3/5E703E3A703E18EB/data/mm_barcode/"
bags = ['Images']
file_list = []
for bag in bags:
    path = os.path.join(datapath, bag, 'Annotations')

    for (dirpath, dirnames, filenames) in os.walk(path):
        #print(filenames)
        for filename in filenames:
            mask_dir = dirpath.replace("Annotations", "Masks")
            if not os.path.exists(mask_dir):
                os.makedirs(mask_dir)
            mask_name = filename.split(".")[0] + ".png"
            img_path = os.path.join(dirpath, filename)
            mask_path = os.path.join(mask_dir, mask_name)
            #mask = xml2mask(img_path)
            #cv2.imwrite(mask_path, mask,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
            file_list.append([img_path[len(datapath):], mask_path[len(datapath):]])

## Generate CSV file

In [14]:
with open(datapath + 'test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for file in file_list:
        jpg_file = file[0].replace("Annotations", "Images").replace(".xml", ".jpg")
        writer.writerow([jpg_file, file[1]])